### Loading the data

In [1]:
import pandas as pd
from pymongo import MongoClient 
import pickle
import matplotlib.pyplot as plt



In [2]:
client = MongoClient()
db = client.SpotifyData



In [ ]:
db.list_collection_names()



In [ ]:
def read_mongo(db, collection, query={}, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df



In [3]:
try:
    data_df=pickle.load(open('spotify_df.pkl', "rb"))
except(OSError) as e:
    data_df = read_mongo(db, 'track_features')
    pickle.dump(data_df, open("spotify_df.pkl" ,"wb"))


In [4]:
data_df.head()



,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,4llunZfVXv3NvUzXVB3VVL,Maria,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],8,1,False,0.277,...,0.0883,0.00694,0.000054,0.1880,0.618,172.848,228093,4.0,1999,1999-11-02
1,21Mq0NzFoVRvOmLTOnJjng,Voice of the Voiceless,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],9,1,False,0.441,...,0.0440,0.01950,0.006840,0.1500,0.418,83.371,151573,4.0,1999,1999-11-02
2,2LXPNLSMAauNJfnC58lSqY,Born of a Broken Man,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],6,1,False,0.298,...,0.0727,0.05380,0.001520,0.2010,0.194,148.282,280960,4.0,1999,1999-11-02
3,7o2Razxnl33Jrgz4PpMYNa,Ashes In the Fall,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],11,1,True,0.456,...,0.0982,0.00520,0.000004,0.0595,0.656,140.629,277267,4.0,1999,1999-11-02
4,3Hxu3Ey1TV9mZ5XqMXckiS,War Within a Breath,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],12,1,False,0.399,...,0.1430,0.00442,0.031400,0.3670,0.174,98.464,216427,4.0,1999,1999-11-02


### EDA

QQ:
1) Music feature trends annually?
2) Track length trends 
3) Explicitness trends
4) Release trends based on release month? 



In [ ]:
data_df.columns



In [ ]:
data_df = data_df.dropna()
data_df.info()



In [ ]:
data_df = data_df[data_df.year!=0]

In [ ]:
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
data_df.hist(ax=ax)

In [ ]:
plt.scatter(data_df.year, data_df.danceability)


In [ ]:
plt.scatter(data_df.year, data_df.energy)

In [ ]:
plt.scatter(data_df.year, data_df.key)

In [ ]:
plt.scatter(data_df.year, data_df.loudness)

In [ ]:
plt.scatter(data_df.year, data_df.speechiness)

In [ ]:
plt.scatter(data_df.year, data_df.acousticness)

In [ ]:
plt.scatter(data_df.year, data_df.instrumentalness)

In [ ]:
plt.scatter(data_df.year, data_df.liveness)

In [ ]:
plt.scatter(data_df.year, data_df.valence)

In [ ]:
plt.scatter(data_df.year, data_df.tempo)

#### Future Steps:
- For Streamlit: based on year - have trends for different features 
- add Genre and popularity from the Spotify API for each track/artist 
- Trends on most popular artists, top artist in each category: danceable, acoustic, instumental, tempo

In [ ]:
with open("credentials.txt") as f:
        cred_ls = f.readlines()
        client = cred_ls[0][:-1]
        sec = cred_ls[1]

In [ ]:
sec

In [5]:
#Trying out Spotipy 

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '69748a4cc75f442cbc4ba4976b658e96'
secret = '2d8774e9e152410c9fb90c15613cc130'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
row = data_df.iloc[2344]
row

id                                             1JHQ6Lsi2bjwGRgpKh9i7h
name                   Scottish Fantasy, Op. 46: II. Allegro - Adagio
album               Bruch: Complete Works for Violin & Orchestra, ...
album_id                                       50ISCnrnkLOtCVNFkY7kE3
artists             ['Max Bruch', 'Antje Weithaas', 'NDR Radiophil...
artist_ids          ['0521x50ZcNqqT1fKMJg5c5', '6ZDnLZicoCvV3R7Gj7...
track_number                                                        6
disc_number                                                         1
explicit                                                        False
danceability                                                    0.368
energy                                                         0.0859
key                                                                 7
loudness                                                      -20.603
mode                                                                1
speechiness         

In [ ]:
track_pop = sp.track(row.id)["popularity"]

artist_ids = row.artist_ids.strip('][').split(', ')
artist_pop = []
genres = []

for ids in artist_ids:
    artist = ids.strip("''")
    artist_pop.append(sp.artist(artist)["popularity"])
    
    genre = sp.artist(artist)["genres"]
    
    if genre:
        genres.append(genre)
    else: 
        genres.append("unknown")


In [7]:
response = sp.track(row.id)

KeyboardInterrupt: 

In [ ]:
response